## E. ML Regression models, apartments

In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pickle
import json


In [114]:
Houses = pd.read_csv('DEF_house.csv', sep=',', na_values='None')
Aparts = pd.read_csv('DEF_apart.csv', sep=',', na_values='None')

In [115]:
Aparts = Aparts.drop('Unnamed: 0', axis=1)

In [116]:
Aparts.columns

Index(['price', 'Commune', 'rank_city_pr', 'house_area', 'number_of_rooms',
       'terrace', 'terrace_area', 'fully_equipped_kitchen', 'open_fire',
       'SB_as new', 'SB_just renovated', 'SB_good', 'SB_to be done up',
       'SB_to renovate', 'SB_to restore'],
      dtype='object')

In [117]:
Aparts

,price,Commune,rank_city_pr,house_area,number_of_rooms,terrace,terrace_area,fully_equipped_kitchen,open_fire,SB_as new,SB_just renovated,SB_good,SB_to be done up,SB_to renovate,SB_to restore
0,45000,LIÈGE,84.0,47.0,1,1,2.0,0,0,0,0,1,0,0,0
1,25000,LIÈGE,84.0,125.0,1,1,8.0,1,1,1,0,0,0,0,0
2,25000,LIÈGE,84.0,142.0,3,1,4.0,1,0,1,0,0,0,0,0
3,20000,COURCELLES,41.0,115.0,1,0,0.0,1,0,0,0,1,0,0,0
4,50000,SCHAERBEEK,404.0,96.0,2,0,0.0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19844,950000,KNOKKE-HEIST,570.0,157.0,2,0,0.0,1,0,0,0,1,0,0,0
19845,945000,TONGEREN,201.0,164.0,3,0,0.0,1,0,1,0,0,0,0,0
19846,950000,DURBUY,194.0,111.0,2,1,115.0,0,0,0,0,0,0,0,0
19847,950000,BRUGGE,468.0,182.0,2,0,0.0,1,0,0,1,0,0,0,0


In [118]:
forImputation_a=Aparts.drop('Commune',axis=1)

In [119]:
forImputation_a.isna().sum()

price                        0
rank_city_pr                 0
house_area                   0
number_of_rooms              0
terrace                      0
terrace_area              3429
fully_equipped_kitchen       0
open_fire                    0
SB_as new                    0
SB_just renovated            0
SB_good                      0
SB_to be done up             0
SB_to renovate               0
SB_to restore                0
dtype: int64

In [120]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()
imputed_data_a = imputer.fit_transform(forImputation_a)

In [121]:
imputed_data_a.shape

(19849, 14)

In [122]:
Aparts_I = pd.DataFrame(data=imputed_data_a)

In [123]:
Aparts_I.columns = ['price', 'rank_city_pr', 'house_area', 'number_of_rooms',
       'terrace', 'terrace_area', 'fully_equipped_kitchen', 'open_fire',
       'SB_as new', 'SB_just renovated', 'SB_good', 'SB_to be done up',
       'SB_to renovate', 'SB_to restore']

In [124]:
Aparts_I.isna().sum()

price                     0
rank_city_pr              0
house_area                0
number_of_rooms           0
terrace                   0
terrace_area              0
fully_equipped_kitchen    0
open_fire                 0
SB_as new                 0
SB_just renovated         0
SB_good                   0
SB_to be done up          0
SB_to renovate            0
SB_to restore             0
dtype: int64

In [125]:
X_a=Aparts_I.drop('price',axis=1) 
y_a=Aparts_I['price'] 

In [126]:
X_a_poly = X_a
X_a_poly['rank_city_pr2'] = X_a_poly['rank_city_pr']**2
X_a_poly['rank_city_pr3'] = X_a_poly['rank_city_pr']**3
X_a_poly.columns

Index(['rank_city_pr', 'house_area', 'number_of_rooms', 'terrace',
       'terrace_area', 'fully_equipped_kitchen', 'open_fire', 'SB_as new',
       'SB_just renovated', 'SB_good', 'SB_to be done up', 'SB_to renovate',
       'SB_to restore', 'rank_city_pr2', 'rank_city_pr3'],
      dtype='object')

In [127]:
X_a_poly = X_a_poly[['rank_city_pr', 'rank_city_pr2', 'rank_city_pr3', 'house_area', 'number_of_rooms', 'terrace',
       'terrace_area', 'fully_equipped_kitchen', 'open_fire', 'SB_as new',
       'SB_just renovated', 'SB_good', 'SB_to be done up', 'SB_to renovate',
       'SB_to restore']]
X_a_poly

,rank_city_pr,rank_city_pr2,rank_city_pr3,house_area,number_of_rooms,terrace,terrace_area,fully_equipped_kitchen,open_fire,SB_as new,SB_just renovated,SB_good,SB_to be done up,SB_to renovate,SB_to restore
0,84.0,7056.0,592704.0,47.0,1.0,1.0,2.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,84.0,7056.0,592704.0,125.0,1.0,1.0,8.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,84.0,7056.0,592704.0,142.0,3.0,1.0,4.000000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,41.0,1681.0,68921.0,115.0,1.0,0.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,404.0,163216.0,65939264.0,96.0,2.0,0.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19844,570.0,324900.0,185193000.0,157.0,2.0,0.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19845,201.0,40401.0,8120601.0,164.0,3.0,0.0,0.000000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
19846,194.0,37636.0,7301384.0,111.0,2.0,1.0,115.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19847,468.0,219024.0,102503232.0,182.0,2.0,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [128]:
# X_h

In [129]:
# y_h

In [130]:
X_train,X_test,y_train,y_test = train_test_split(X_a_poly,y_a, test_size =0.2)

In [131]:
Mregressor = LinearRegression(normalize=True)
Mregressor.fit(X_train, y_train)

LinearRegression(normalize=True)

In [132]:
Mregressor.score(X_train, y_train)

0.6072859997570188

In [133]:
Mregressor.score(X_test, y_test)

0.6084272595169331

In [134]:
Mregressor.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True}

In [135]:
Mregressor.coef_

array([ 1.49787943e+03, -5.85426551e+00,  7.39598881e-03,  1.51039157e+03,
        1.39287159e+04, -7.08895979e+03,  1.62547495e+03, -3.42057199e+03,
        1.98404485e+04,  1.83131021e+04,  1.47951726e+04, -2.07410641e+04,
       -4.99081778e+04, -9.06109751e+04, -1.24843124e+05])

In [136]:
type(X_train)

pandas.core.frame.DataFrame

In [137]:
X_train.iloc[0,:]

rank_city_pr                    537.0
rank_city_pr2                288369.0
rank_city_pr3             154854153.0
house_area                       65.0
number_of_rooms                   2.0
terrace                           0.0
terrace_area                      0.0
fully_equipped_kitchen            0.0
open_fire                         0.0
SB_as new                         0.0
SB_just renovated                 0.0
SB_good                           1.0
SB_to be done up                  0.0
SB_to renovate                    0.0
SB_to restore                     0.0
Name: 9414, dtype: float64

In [138]:
X_train.iloc[0,:].values.reshape(1,X_train.shape[1])

array([[5.37000000e+02, 2.88369000e+05, 1.54854153e+08, 6.50000000e+01,
        2.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [139]:
X_train.iloc[0,:].shape

(15,)

In [140]:
Mregressor.predict(X_train.iloc[0,:].values.reshape(1,X_train.shape[1]))

array([310603.82969328])

In [141]:
y_train.iloc[0]

225000.0

In [142]:
pickle.dump(Mregressor, open('Apart_regression', 'wb'))


In [143]:
essai = pickle.load(open('Apart_regression', 'rb'))


In [144]:
essai.predict(X_train.iloc[66,:].values.reshape(1,X_train.shape[1]))

array([273322.63448156])

In [145]:
type(X_train.iloc[66,:].values.reshape(1,X_train.shape[1]))

numpy.ndarray

In [146]:
y_train.iloc[66]

322500.0

In [147]:
X_train.columns

Index(['rank_city_pr', 'rank_city_pr2', 'rank_city_pr3', 'house_area',
       'number_of_rooms', 'terrace', 'terrace_area', 'fully_equipped_kitchen',
       'open_fire', 'SB_as new', 'SB_just renovated', 'SB_good',
       'SB_to be done up', 'SB_to renovate', 'SB_to restore'],
      dtype='object')

In [148]:
Aparts.describe().T

,count,mean,std,min,25%,50%,75%,max
price,19849.0,288219.931483,150442.318666,2500.0,193000.0,249000.0,335000.0,950000.0
rank_city_pr,19849.0,324.819386,159.199740,1.0,193.0,329.0,473.0,580.0
house_area,19849.0,99.861252,44.352131,11.0,75.0,92.0,115.0,700.0
number_of_rooms,19849.0,2.013250,0.842299,0.0,2.0,2.0,2.0,13.0
terrace,19849.0,0.677566,0.467420,0.0,0.0,1.0,1.0,1.0
terrace_area,16420.0,10.358465,15.050800,0.0,0.0,6.0,14.0,115.0
fully_equipped_kitchen,19849.0,0.690463,0.462314,0.0,0.0,1.0,1.0,1.0
open_fire,19849.0,0.015416,0.123205,0.0,0.0,0.0,0.0,1.0
SB_as new,19849.0,0.393874,0.488620,0.0,0.0,0.0,1.0,1.0
SB_just renovated,19849.0,0.050632,0.219251,0.0,0.0,0.0,0.0,1.0


In [205]:
def make_apart_R (cp, house_area=100, numb_rooms=2, terrace=1, terrace_area=10, equip_kitch=1, fire=0, Sas_new=1, 
                  Sjust_renov=0, Sgood=0, Sto_refresh=0, Sto_renov=0, Sto_restor=0):
    with open('CP_to_city.json', 'r') as fcp:
        cp_to_city = json.load(fcp)
        city = cp_to_city['Commune principale'][str(cp)]
    with open('City_to_rank.json', 'r') as fci:
        city_to_rank = json.load(fci)
        rank = city_to_rank[str(city)]
    features = np.array([[rank, rank**2, rank**3,house_area, numb_rooms, terrace, terrace_area, equip_kitch, fire, 
                          Sas_new, Sjust_renov, Sgood, Sto_refresh, Sto_renov, Sto_restor]])
    Apart_model = pickle.load(open('Apart_regression', 'rb'))
    return Apart_model.predict(features)[0]

In [206]:
make_apart_R (7850, 99, 2, 0, 0, 1, 0, 1,0,0,0,0,0)

302349.75025980524

### House and apartment variables

In [151]:
# apartment_variables = (cp, house_area=100, numb_rooms=2, terrace=1, terrace_area=10, equip_kitch=1, fire=0, Sas_new=1, 
#                   Sjust_renov=0, Sgood=0, Sto_refresh=0, Sto_renov=0, Sto_restor=0)
# house_variables = (cp, house_area=196, surf_land=753, numb_facades=3, numb_rooms=3, garden=1, 
#                   terrace=0, terrace_area=0, equip_kitch=1, fire=0, pool=0, Sas_new=0, Sjust_renov=0, 
#                   Sgood=1, Sto_refresh=0, Sto_renov=0, Sto_restor=0)

In [194]:
variables = {"building":"house", "Postalcode":7850, "house_area":196, "surf_land":125,"numb_facades":4, "numb_rooms":3, "garden":1, 
 "terrace":0, "terrace_area":10, "equip_kitch":1, "fire":1, "pool":1, "Sas_new":1, "Sjust_renov":0,
 "Sgood":0, "Sto_refresh":0, "Sto_renov":0, "Sto_restor":0}

In [195]:
variables["building"] == 'house'

True

In [196]:
del variables["building"]

In [197]:
newlist = list(variables.values())

In [198]:
X_train.iloc[66,:].values.reshape(1,X_train.shape[1]).shape

(1, 15)

In [199]:
newlist[0:13]


[7850, 196, 125, 4, 3, 1, 0, 10, 1, 1, 1, 1, 0]

In [78]:
for n, i in enumerate(newlist):
    if i == True:
        newlist[n] = 1
    elif i == False: 
        newlist[n] = 0


In [94]:
newlist[0:13]

[7830, 196, 125, 4, 3, 1, 0, 10, 1, 1, 1, 1, 0]

In [95]:
arr = np.array([newlist[0:13]])


In [96]:
arr

array([[7830,  196,  125,    4,    3,    1,    0,   10,    1,    1,    1,
           1,    0]])

In [98]:
arr.shape

(1, 13)

In [ ]:
7850, 99, 2, 0, 0, 1, 0, 1,0,0,0,0,0

In [70]:
list(variables.values())

TypeError: list indices must be integers or slices, not str

In [85]:
newlist.as

'[7830, 196, 125, 4, 3, 1, 0, 10, 1, 1, 1, 1, 0, 0, 0, 0, 0]'

In [84]:
newlist[0]

7830

In [202]:
make_apart_R(7850, 99, 2, 0, 0, 1, 0, 1,0,0,0,0,0)

(array([302349.75025981]), 455.0)

In [187]:
make_apart_R(newlist[0], newlist[1], newlist[2], newlist[3], newlist[4], newlist[5], newlist[6], 
             newlist[7], newlist[8], newlist[9], newlist[10], newlist[11], newlist[12])

(7850, 196, 125, 4, 3, 1, 0, 10, 1, 1, 1, 1, 0)
{}


In [180]:
type(newlist)

list

In [188]:
def make_apart_R(*args):
  print(args)
  

In [189]:
make_apart_R(newlist[0:13])

([7850, 196, 125, 4, 3, 1, 0, 10, 1, 1, 1, 1, 0],)


SyntaxError: unexpected EOF while parsing (<ipython-input-193-bf2d7a031c3e>, line 1)